In [1]:
import os
import warnings

import h5py
import numpy as np

from ipynb.fs.full.utils import resetRandom

resetRandom()
from ipynb.fs.full.dnn import dnn

from ipynb.fs.full.cnn import cnn
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
from ipynb.fs.full.utils import plot_roc_curve ,CLASS_NAMES
from ipynb.fs.full.utils import plot_conf_matrix
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import load_model
import tensorflow
import tensorflow.keras as keras
import numpy as np
warnings.filterwarnings("ignore")

In [2]:
class AccuracyCallback(keras.callbacks.Callback):
    def __init__(self, threshold):
        super(AccuracyCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') > self.threshold and logs.get('val_accuracy') > 0.96:
            print("\nReached val_accuracy threshold of {}%, so cancelling training!".format(self.threshold*100))
            self.model.stop_training = True

In [3]:
features_train = {'Facenet': {'train_x': None, 'train_y': None}
                 }

features_test = {'Facenet': {'test_x': None, 'test_y': None}
                }

classifiers = {'Facenet': [cnn,dnn]}
train_data = {'Facenet': {'epochs': {'dnn': (150, 32), 'cnn': (150, 32)}}}

features_dir = 'features'
classifier_dir = 'classifiers'
results_dir = 'results'
os.makedirs(classifier_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

In [4]:
for feature in features_train.keys():
    fe_path = os.path.join(features_dir, '{0}_{1}.h5'.format(feature, 'features_train'))
    feats = h5py.File(fe_path, 'r')
    fes = np.array(feats.get('features'))
    lb_path = os.path.join(features_dir, '{0}_{1}.h5'.format(feature, 'labels_train'))
    labels = h5py.File(lb_path, 'r')
    lbs = np.array(labels.get('labels'))
    features_train[feature]['train_x'] = fes
    features_train[feature]['train_y'] = lbs

In [5]:
for feature in features_test.keys():
    fe_path = os.path.join(features_dir, '{0}_{1}.h5'.format(feature, 'features_test'))
    print(fe_path)
    feats = h5py.File(fe_path, 'r')
    fes = np.array(feats.get('features'))
    lb_path = os.path.join(features_dir, '{0}_{1}.h5'.format(feature, 'labels_test'))
    labels = h5py.File(lb_path, 'r')
    lbs = np.array(labels.get('labels'))
    features_test[feature]['test_x'] = fes
    features_test[feature]['test_y'] = lbs

classifier_out = {'Feature': [], 'Classifier': [], 'Accuracy': [],
                  'Precision': [], 'Recall': [],
                  'F1-Score': []}

features/Facenet_features_test.h5


In [6]:
def get_best_dimension(x,array_size,num_dimensions):
    # Find the dimensions that result in the closest match to the target size
    best_dimensions = None
    min_difference = float('inf')

    for dim1 in range(1, int(np.power(array_size, 1/num_dimensions)) + 1):
        if array_size % dim1 == 0:
            dim_remainder = array_size // dim1
            dimensions = [dim1]
            for _ in range(num_dimensions - 1):
                dimensions.append(dim_remainder)
                dim_remainder = dim_remainder // dim1
            difference = max(dimensions) - min(dimensions)
            if difference < min_difference:
                min_difference = difference
                best_dimensions = list(dimensions)

    d=[]
    d.append(x)
    d.append(dimensions[0])
    d.append(dimensions[1])
    d.append(3)
    return tuple(d)  ## returning the expected dimension

In [7]:
def get_test_data(feature,classifier):
    print("IN TEST")
    x = features_test[feature]['test_x']
    y = features_test[feature]['test_y']
    y_cat = to_categorical(y)
    x_data = x
    print(x.shape)
    x_shape = x.shape[1]
    c_name = classifier.__name__
    if c_name == 'cnn':
        array_size = x.shape[1] // 3 #need shape with RGB channels 
        get_shape = get_best_dimension(x_data.shape[0],array_size,2)
        x_data = np.reshape(x, get_shape)
        x_shape = x_data[0].shape
    print('[INFO] Test Feature :: {0} Data Shape :: {1}'.format(feature, x_data.shape))
    return x_data,y_cat

<h3>Running the custom CNN and DNN model with extracted features</h3>

In [8]:
for feature in list(features_train.keys()):
    x = features_train[feature]['train_x']
    y = features_train[feature]['train_y']
    y_cat = to_categorical(y)
    print(set(y))

    for classifier in classifiers[feature]:
        x_data = x
        print(x.shape)
        x_shape = x.shape[1]
        c_name = classifier.__name__
        if c_name == 'cnn':
            array_size = x.shape[1] // 3 #need shape with RGB channels 
            get_shape = get_best_dimension(x_data.shape[0],array_size,2)
            x_data = np.reshape(x, get_shape)
            x_shape = x_data[0].shape
        print('[INFO] Feature :: {0} Data Shape :: {1}'.format(feature, x_data.shape))
        classifier_path = os.path.join(classifier_dir, '{0}_{1}.h5'.format(feature, c_name))
        print(classifier_path)
        resetRandom()
        print('[INFO] Feature :: {0} Classifier :: {1}'.format(feature, c_name))
        x_test, y_test = get_test_data(feature,classifier)
        classifier = classifier(x_shape)
        checkpoint = ModelCheckpoint(classifier_path, verbose=0,
                             monitor='val_loss',save_best_only=True, mode='auto')
        accuracy_callback = AccuracyCallback(threshold=0.95)
        
       
        print(classifier.summary())
        classifier.fit(x_data, y_cat, epochs=train_data[feature]['epochs'][c_name][0],
                       batch_size=train_data[feature]['epochs'][c_name][1],
                       validation_data=(x_test, y_test), callbacks=[checkpoint,accuracy_callback])

        train_pred = np.argmax(classifier.predict(x_test), axis=1)
       
        y = features_test[feature]['test_y']
        train_res = classification_report(y, train_pred, output_dict=True)
        train_cm = confusion_matrix(y, train_pred)
        plot_conf_matrix(train_cm, feature, c_name,
                         CLASS_NAMES, 'results/{0}_{1}.png'.format(feature, c_name))
        plot_roc_curve(y, train_pred, feature, c_name, results_dir)
        classifier_out['Feature'].append(feature)
        classifier_out['Classifier'].append(c_name)
        classifier_out['Accuracy'].append('{0}%'.format(
            round(train_res['accuracy'] * 100, 2)
        ))
        classifier_out['Precision'].append('{0}%'.format(
            round(train_res['macro avg']['precision'] * 100, 2)
        ))
        classifier_out['Recall'].append('{0}%'.format(
            round(train_res['macro avg']['recall'] * 100, 2)
        ))
        classifier_out['F1-Score'].append('{0}%'.format(
            round(train_res['macro avg']['f1-score'] * 100, 2)
        ))
pd.DataFrame.from_dict(classifier_out).to_csv('results/classification_results_2.csv', index=None)


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
(450, 15360)
[INFO] Feature :: Facenet Data Shape :: (450, 64, 80, 3)
classifiers/Facenet_cnn.h5
[INFO] Feature :: Facenet Classifier :: cnn
IN TEST
(50, 15360)
[INFO] Test Feature :: Facenet Data Shape :: (50, 64, 80, 3)
Model: "cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 78, 64)        1792      
                                                                 
 activation (Activation)     (None, 62, 78, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 39, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 37, 64)        36928     
                                                                 
 activat

Epoch 95/150
15/15 [==============================] - 3s 182ms/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 1.9028 - val_accuracy: 0.6600
Epoch 96/150
15/15 [==============================] - 3s 182ms/step - loss: 0.0073 - accuracy: 1.0000 - val_loss: 1.7481 - val_accuracy: 0.6800
Epoch 97/150
15/15 [==============================] - 3s 182ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 1.7835 - val_accuracy: 0.7000
Epoch 98/150
15/15 [==============================] - 3s 182ms/step - loss: 0.0065 - accuracy: 1.0000 - val_loss: 1.7830 - val_accuracy: 0.6600
Epoch 99/150
15/15 [==============================] - 3s 182ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 1.7471 - val_accuracy: 0.7000
Epoch 100/150
15/15 [==============================] - 3s 182ms/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 1.8360 - val_accuracy: 0.6600
Epoch 101/150
15/15 [==============================] - 3s 182ms/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 1.7709 - val_accurac

_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 1024)              15729664  
                                                                 
 dense_3 (Dense)             (None, 512)               524800    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                2570      
                                                                 
Total params: 16,388,362
Trainable params: 16,388,362
Non-trainable params: 

Epoch 50/150
15/15 [==============================] - 1s 52ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 2.0262 - val_accuracy: 0.7000
Epoch 51/150
15/15 [==============================] - 1s 52ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 2.2276 - val_accuracy: 0.6800
Epoch 52/150
15/15 [==============================] - 1s 52ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 2.2364 - val_accuracy: 0.6800
Epoch 53/150
15/15 [==============================] - 1s 53ms/step - loss: 7.5469e-04 - accuracy: 1.0000 - val_loss: 2.2383 - val_accuracy: 0.7000
Epoch 54/150
15/15 [==============================] - 1s 53ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 2.2383 - val_accuracy: 0.7000
Epoch 55/150
15/15 [==============================] - 1s 53ms/step - loss: 5.7955e-04 - accuracy: 1.0000 - val_loss: 2.2410 - val_accuracy: 0.7000
Epoch 56/150
15/15 [==============================] - 1s 53ms/step - loss: 8.1473e-04 - accuracy: 1.0000 - val_loss: 2.2264 - val_accu

Epoch 107/150
15/15 [==============================] - 1s 52ms/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 2.5035 - val_accuracy: 0.6600
Epoch 108/150
15/15 [==============================] - 1s 53ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 2.4006 - val_accuracy: 0.6600
Epoch 109/150
15/15 [==============================] - 1s 52ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 2.3666 - val_accuracy: 0.6600
Epoch 110/150
15/15 [==============================] - 1s 52ms/step - loss: 6.1766e-04 - accuracy: 1.0000 - val_loss: 2.3635 - val_accuracy: 0.6800
Epoch 111/150
15/15 [==============================] - 1s 53ms/step - loss: 5.9664e-04 - accuracy: 1.0000 - val_loss: 2.3802 - val_accuracy: 0.6600
Epoch 112/150
15/15 [==============================] - 1s 52ms/step - loss: 0.0039 - accuracy: 0.9978 - val_loss: 2.4803 - val_accuracy: 0.6600
Epoch 113/150
15/15 [==============================] - 1s 53ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 3.7585 - val_a

<Figure size 640x480 with 0 Axes>